# Config and Setup

In [ ]:
!pip install imblearn pyyaml h5py keras tensorflow numpy xgboost yellowbrick seaborn

from enum import Enum
import pandas as pd
import numpy as np
import seaborn
import pickle
import time
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
import time
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# For evaluation
from sklearn.metrics import confusion_matrix
from yellowbrick.model_selection import LearningCurve
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

# Classification labels
TARGET_LABELS = ['On Time', 'Behind']

# Available locations
class Location(Enum):
    OSLO = "OSL"
    TRONDHEIM = "TRD"

LOCATION = Location.OSLO

# Data balance config
class SamplingStrategy(Enum):
    OVERSAMPLING = 1
    UNDERSAMPLING = 2
    NONE = 3

# Use fixed randomseed, so same datasplits are used for each run?
# Only use this when you want a final reproducible result
USE_FIXED_RANDOMSEED = True
SEED = 64 if USE_FIXED_RANDOMSEED else int(time.time())


SVM_MODEL_PATH = "../models/svm"          # Models for SVM
KNN_MODEL_PATH = "../models/knn"          # Models for KNN
MLP_MODEL_PATH = "../models/ann"          # Models for ANN
XGBOOST_MODEL_PATH = "../models/xgboost"  # Models for XGBoost

SVM_VISUALIZATION_PATH = "../visualizations/models/svm"          # Visualizations for SVM
KNN_VISUALIZATION_PATH = "../visualizations/models/knn"          # Visualizations for KNN
MLP_VISUALIZATION_PATH = "../visualizations/models/ann"          # Visualizations for ANN
XGBOOST_VISUALIZATION_PATH = "../visualizations/models/xgboost"  # Visualizations for XGBoost

# Set up folder structure
for dir in [
    SVM_MODEL_PATH, KNN_MODEL_PATH, ANN_MODEL_PATH, XGBOOST_MODEL_PATH, 
    SVM_VISUALIZATION_PATH, KNN_VISUALIZATION_PATH, ANN_VISUALIZATION_PATH, XGBOOST_VISUALIZATION_PATH
]:
    os.makedirs(dir, exist_ok=True)

NUM_FOLDS = 5 # K-value for Cross-Validation
SK_LEARNING_CURVE_SPACE = np.linspace(0.3, 1.0, 10)


    

## Helper functions

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline, make_pipeline
from collections import Counter


def create_pipeline(model, sampling_strategy, _y):
    '''
    Creates a imblearn datbalancing pipeline, such that databalancing can be combined with CV,
    without balancing test-data.
    When .fit() method is called on the pipeline, 
    all training data will go through the chosen sampling_strategy
    @model must be a model that implements fit()
    '''
    if sampling_strategy == SamplingStrategy.NONE:
        balancer = 'passthrough'
    elif sampling_strategy == SamplingStrategy.UNDERSAMPLING:
        databalancing_stats(_y, sampling_strategy)        
        balancer = RandomUnderSampler(random_state=SEED)
    elif sampling_strategy == SamplingStrategy.OVERSAMPLING:
        databalancing_stats(_y, sampling_strategy)
        balancer = SMOTE(random_state=SEED, n_jobs=-1)
    return make_pipeline(balancer, model)

def databalancing_stats(_y, sampling_strategy):
    '''
    Currently only works for binary problem.
    This is only to give user an idea of what will happen to the
    training data in the training pipeline, if databalancing is used
    '''
    original_count = Counter(_y)
    print('Following statistics is to get an idea about what will be done to the training-set(s):')
    print('Targets before databalancing:', original_count)
    if sampling_strategy == SamplingStrategy.OVERSAMPLING:
        max_key = max(original_count, key=original_count.get)
        new_count = '0: %s 1: %s' % (original_count[max_key], original_count[max_key])
    else:
        min_key = min(original_count, key=original_count.get)
        new_count = '0: %s 1: %s' % (original_count[min_key], original_count[min_key])
    print('Targets after databalancing: %s \n' % new_count)

# Synthetic sugar for models wrapped in pipelines
def pipeline_search_params(model_name, params):
    return {model_name+'__' + key: params[key] for key in params}

def create_model_name():
  now = datetime.now()
  return '%s-%s' % (LOCATION.value, now.strftime("%d-%m-%Y-%H%M%S"))

def evaluate_model(X, y, model,  gpu_mode=False):
    """
    Evaulates the model using k-fold kross validation.
    ---
    returns two dataframes: 
    1 => scores for all runs
    2 => average scores
    """
    cv = StratifiedKFold(n_splits=NUM_FOLDS, random_state=SEED, shuffle=True)
    N_JOBS = 1 if gpu_mode else -1

    scores = cross_validate(model, X, y, scoring=['f1_macro', 'f1_micro'], cv=cv, n_jobs=N_JOBS)

    averages = {}
    for key, val in scores.items(): 
        averages[key] = val.mean()

    return pd.DataFrame(scores), averages

def confusionMatrix(_X, _y, training_pipeline, figPath = None):
    X_train, X_test, y_train, y_test = train_test_split(_X, _y, test_size=0.2, random_state=SEED)
    model = training_pipeline.fit(X_train, y_train)
    predicted = model.predict(X_test)
    data = confusion_matrix(y_test, predicted)

    # Below is from https://onestopdataanalysis.com/confusion-matrix-python/
    seaborn.set(color_codes=True)
    plt.figure(1, figsize=(9, 6))
 
    plt.title("Confusion Matrix")
 
    seaborn.set(font_scale=1.4)
    ax = seaborn.heatmap(data, annot=True, fmt='g', cmap="YlGnBu", cbar_kws={'label': 'Scale'})
 
    ax.set_xticklabels((TARGET_LABELS[0], TARGET_LABELS[1]))
    ax.set_yticklabels((TARGET_LABELS[0], TARGET_LABELS[1]))
 
    ax.set(ylabel="True Label", xlabel="Predicted Label")
    
    if not figPath == None:
        plt.savefig(figPath+"/confusion.png")

    plt.show()


def plot_learning_curve(X, y, model, figPath = None):
    cv = StratifiedKFold(n_splits=NUM_FOLDS, random_state=SEED, shuffle=True)

    visualizer = LearningCurve(model, 
        cv=cv, 
        scoring='f1_weighted',
        train_sizes=SK_LEARNING_CURVE_SPACE,
        n_jobs=-1
    )
    visualizer.fit(X, y)

    if figPath == None:
        visualizer.show()
    else:
        visualizer.show(outpath=figPath+"/learning.png")

# Plotting Keras learning curve
# From: https://stackabuse.com/python-for-nlp-multi-label-text-classification-with-keras/
def plot_learning_curve_keras(history):
  # Plot accuracy
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])

  plt.title('Model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train','validation'], loc='upper left')
  plt.show()

  # Plot loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])

  plt.title('Model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train','validation'], loc='upper left')
  plt.show()

# timing can be used to time a code block.
# Example:
#   with timing():
#      expensive_operation()
#
#   > Finished in _ seconds.
class timing:
    def __enter__(self):
        self.start_time = time.time()
    def __exit__(self, type, exc, tb):
        print('Finished in {:.3f} seconds'.format(time.time() - self.start_time))

## Search

In [ ]:
def gridParamSearch(X, y, model, param_dist):
    print("[GRID SEARCH (CV)] Finding best model using parameters:")
    print(param_dist, end="\n\n")

    # run grid search
    search = GridSearchCV(model, param_grid=param_dist, n_jobs=-1, scoring="f1_macro")
    start_time = time.time()
    search.fit(X, y)

    print("GridSearchCV took %.2f seconds for %d candidate parameter settings." % (time.time() - start_time, len(search.cv_results_['params'])))

    print("\nBest candidate:")
    print("Params", search.best_params_)
    print("Score:", search.best_score_)

def randomParamSearch(X, y, model, num_searches, param_dist):
    print("[RANDOM SEARCH (CV)] Finding best model using parameters:")
    print(param_dist, end="\n\n")

    # run grid search
    search = RandomizedSearchCV(model, param_distributions=param_dist, n_jobs=-1, scoring="f1_macro", n_iter=num_searches)
    start_time = time.time()
    search.fit(X, y)

    print("RandomizedSearchCV took %.2f seconds for %d candidates parameter settings." % ((time.time() - start_time), num_searches))

    print("\nBest candidate:")
    print("Params", search.best_params_)
    print("Score:", search.best_score_)


    

# Load data

In [ ]:
df = pd.read_csv("../data/clean/%s.csv" % LOCATION.value)

# Select x random-seeded samples from the full dataset
df = df.sample(500000, random_state=SEED)
df

## Pre-process data

### Feature selection

In [ ]:
from sklearn.preprocessing import LabelEncoder

y = df["TARGET"]
X = df[[
    'MONTH',
    'DAY_OF_MONTH',
    'WEEKDAY',
    'HOUR',
    'WIND_SPEED', 
    'WIND_DIRECTION', 
    'AIR_TEMPERATURE', 
    'PRECIPITATION', 
    'VISIBILITY'
]]

# Parse string features to numeric values
le = LabelEncoder()
le.fit(df['AIRLINE_IATA'].values)
X.insert(2, "AIRLINE_IATA", le.transform(df['AIRLINE_IATA'].values), True) 

le = LabelEncoder()
le.fit(df['GATE_STAND'].astype(str).values)
X.insert(2, "GATE_STAND", le.transform(df['GATE_STAND'].astype(str).values), True)

le = LabelEncoder()
le.fit(df['INT_DOM_SCHENGEN'].values)
X.insert(2, "INT_DOM_SCHENGEN", le.transform(df['INT_DOM_SCHENGEN'].values), True)

le = LabelEncoder()
le.fit(df['DEP_ARR'].values)
X.insert(2, "DEP_ARR", le.transform(df['DEP_ARR'].values), True) 

le = LabelEncoder()
le.fit(df['TO_FROM'].values)
X.insert(2, "TO_FROM", le.transform(df['TO_FROM'].values), True)

le = LabelEncoder()
le.fit(df['FLIGHT_ID'].values)
X.insert(2, "FLIGHT_ID", le.transform(df['FLIGHT_ID'].values), True)

# print histogram of features
X.hist()

# Convert to numpy arrays
X, y = X.values, y.values

### Baseline score

In [ ]:
from sklearn.metrics import f1_score
import sys
sys.path.insert(1, '../src/pipeline')
# Our custom weighted dice baseline classifier
from baseline import weighted_dice

y_test = train_test_split(X, y, test_size=.2, random_state=SEED)[3]

targets = Counter(y_test)

micros = []
macros = []

# We average the baseline over 10 runs, to get a more stable score
for run in range(10):
    y_baseline_pred = [weighted_dice(targets) for target in y_test]
    f1_micro = f1_score(y_test, y_baseline_pred, average='micro')
    f1_macro = f1_score(y_test, y_baseline_pred, average='macro')
    micros.append(f1_micro)
    macros.append(f1_macro)

micros = np.array(micros)
macros = np.array(macros)
print('Average scores => f1_macro: %.4f \tf1_micro: %.4f' % (macros.mean(), micros.mean()))

# Methods

## SVM

In [ ]:
from sklearn.svm import SVC

# 'Empty' model is declared here and is used on param search,
# it's parameters are set in 'Build optmized model' based on
# the optimal params found in param search cell
svm_pipeline = create_pipeline(SVC(), sampling_strategy=SamplingStrategy.UNDERSAMPLING, _y=y)

### Param Search

In [ ]:
#gridParamSearch(X, y, svm_pipeline, pipeline_search_params('svc', {
#    'C': [1, 2, 5, 10],
#    'gamma': [0.01, 0.001, 0.0001],
#    'class_weight': ['balanced', None],                          
#    'shrinking': [True, False],
#}))
# 1st search used 50k samples and undersampling.
# Best parameters: C=2, class_weight=balanced, gamma=0.0001, shrinking=true => score 0.47

# Next => homing in on numerical values. Discard enumerated params (shringing + class_weight).
#gridParamSearch(X, y, svm_pipeline, pipeline_search_params('svc', {
#    'C': [1.6, 1.85, 2, 2.15, 2.3],
#    'gamma': [0.0001, 0.00012, 0.00008],
#    'class_weight': ['balanced'],                          
#    'shrinking': [True],
#}))
# 2nd search used 50k samples and undersampling.
# Best parameters: C=1.6, class_weight=balanced, gamma=0.00008, shrinking=true => score 0.493

# Next => homing in on numerical values. Discard enumerated params (shringing + class_weight).
gridParamSearch(X, y, svm_pipeline, pipeline_search_params('svc', {
    'C': [1.55, 1.6, 1.65],
    'gamma': [0.00007, 0.00008, 0.00009],
    'class_weight': ['balanced'],                          
    'shrinking': [True],
}))
# 3rd search used 50k samples and undersampling.
# Best parameters: C=1.6, class_weight=balanced, gamma=0.00007, shrinking=true => score 0.494

### Build Optimal Model

In [ ]:
# Best params from grid search
svm_pipeline = create_pipeline(
    SVC(
        C=1.6, 
        class_weight='balanced', 
        gamma=0.00007, 
        shrinking=True
    ),
    sampling_strategy=SamplingStrategy.UNDERSAMPLING, 
    _y=y
)

### Evaulate

In [ ]:
# 'test_f1_macro': 0.523831637237808, 'test_f1_micro': 0.59231#
with timing():
    scores, averages = evaluate_model(X, y, svm_pipeline)
    print(scores)
    print("Averages:", averages)

    confusionMatrix(X, y, svm_pipeline, SVM_VISUALIZATION_PATH)

### Training

In [ ]:
with timing():
    plot_learning_curve(X, y, svm_pipeline, SVM_VISUALIZATION_PATH)

### Save

In [ ]:
model_name = create_model_name()
pickle.dump(svm_pipeline, open('%s/%s' % (SVM_MODEL_PATH, model_name), 'wb'))

## kNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# 'Empty' model is declared here and is used on param search,
# it's parameters are set in 'Build optmized model' based on
# the optimal params found in param search cell
knn_pipeline = create_pipeline(KNeighborsClassifier(1), sampling_strategy=SamplingStrategy.UNDERSAMPLING, _y=y)

### Param Search

In [ ]:

#gridParamSearch(X, y, knn_pipeline, pipeline_search_params('kneighborsclassifier', {
#    'n_neighbors': [1, 10, 50, 100],
#    'weights': ['distance', 'uniform'],
#}))
# 1st search used 500k samples and undersampling.
# Best parameters: K = 10, weights = 'uniform'

# Next step: search the area around k=10
#gridParamSearch(X, y, knn_pipeline, pipeline_search_params('kneighborsclassifier', {
#    'n_neighbors': [6, 8, 10, 12, 14],
#    'weights': ['distance', 'uniform'],
#}))
# 2nd search used 500k samples and undersampling.
# Scored 0.5437. Best parameters: K = 6, weights = 'uniform'

# Next step: search the area around k=6
gridParamSearch(X, y, knn_pipeline, pipeline_search_params('kneighborsclassifier', {
    'n_neighbors': [5, 6, 7],
    'weights': ['distance', 'uniform'],
}))
# 3rd search used 500k samples and undersampling.
# Scored 0.5437. Best parameters: K = 6, weights = 'uniform'

### Build Optimal Model

In [ ]:
# Best params from grid search
knn_pipeline = create_pipeline(
    KNeighborsClassifier(n_neighbors=6, weights="uniform"), 
    sampling_strategy=SamplingStrategy.UNDERSAMPLING, _y=y
)

### Evaulate

In [ ]:
with timing():
    scores, averages = evaluate_model(X, y, knn_pipeline)
    print(scores)
    print("Averages:", averages)

    confusionMatrix(X, y, knn_pipeline, KNN_VISUALIZATION_PATH)

### Training

In [ ]:
with timing():
    plot_learning_curve(X, y, knn_pipeline, KNN_VISUALIZATION_PATH)

### Saving

In [ ]:
model_name = create_model_name()
pickle.dump(knn_pipeline, open('%s/%s' % (KNN_MODEL_PATH, model_name), 'wb'))

## MLP

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from keras.regularizers import l2

l2_value = 0.00001
def create_model():
    model = Sequential()
    model.add(BatchNormalization(input_dim=X.shape[1]))
    model.add(Dense(X.shape[1], activation="relu", kernel_initializer='he_uniform', kernel_regularizer=l2(l2_value)))
    model.add(Dense(256, activation="relu", kernel_initializer='he_uniform', kernel_regularizer=l2(l2_value)))
    model.add(Dropout(0.1))
    model.add(Dense(256, activation="relu", kernel_initializer='he_uniform', kernel_regularizer=l2(l2_value)))
    model.add(Dropout(0.1))
    model.add(Dense(128, activation="relu", kernel_initializer='he_uniform', kernel_regularizer=l2(l2_value)))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

num_epochs = 200
batch_size = 4096

mlp_model = create_model()
print(mlp_model.summary())

### Evaulate

In [ ]:

with timing():
    mlp_model_for_evaluation = KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=batch_size, verbose=0)
    mlp_pipeline = create_pipeline(mlp_model_for_evaluation, sampling_strategy=SamplingStrategy.OVERSAMPLING, _y=y)
    scores, averages = evaluate_model(X, y, mlp_pipeline, gpu_mode=True)
    print('\n\n', scores)
    print("Averages:", averages)

    confusionMatrix(X, y, mlp_pipeline, MLP_VISUALIZATION_PATH)

### Training

In [ ]:
def mlp_databalancing(_X, _y, sampling_strategy):
    if sampling_strategy == SamplingStrategy.UNDERSAMPLING:      
        _X, _y = RandomUnderSampler(random_state=SEED).fit_resample(_X, _y)
    elif sampling_strategy == SamplingStrategy.OVERSAMPLING:
        _X, _y = SMOTE(random_state=SEED, n_jobs=-1).fit_resample(_X, _y)
    return _X, _y

SPLIT_AND_VALIDATE = True # If False, train with all data
SAMPLING_STRATEGY = SamplingStrategy.OVERSAMPLING

with timing():
    # Custom code for databalancing for MLP, since imblearn pipeline don't return keras-history
    if SPLIT_AND_VALIDATE:
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=SEED)
        if SAMPLING_STRATEGY != SamplingStrategy.NONE:
            X_train, y_train = mlp_databalancing(X_train, y_train, SAMPLING_STRATEGY)
        history = mlp_model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val, y_val), verbose=0)
    else:
        X, y = mlp_databalancing(X, y, SAMPLING_STRATEGY)
        history = mlp_model.fit(X, y, epochs=num_epochs, batch_size=batch_size, verbose=0)
    
    plot_learning_curve_keras(history)

### Save

In [ ]:
model_name = create_model_name()
history.save('%s/%s' % (MLP_MODEL_PATH, model_name))

## XGBoost

In [ ]:
from xgboost import XGBClassifier

# 'Empty' model is declared here and is used on param search,
# it's parameters are set in 'Build optmized model' based on
# the optimal params found in param search cell
xgb_pipeline = create_pipeline(XGBClassifier(), sampling_strategy=SamplingStrategy.UNDERSAMPLING, _y=y)

### Param Search

In [ ]:
#gridParamSearch(X, y, xgb_pipeline, pipeline_search_params('xgbclassifier', {
#    'gamma': [0, 5, 10, 20],
#    'learning_rate': [0.1, 0.2, 0.3],                
#    'max_depth': [5, 10, 20, 30],
#}))
# 1st search used 250k samples and undersampling.
# Best parameters: gamma=10, learning_rate=0.1, max_depth=30 => score: 0.6114179052808012

# Next: Focus parameters
gridParamSearch(X, y, xgb_pipeline, pipeline_search_params('xgbclassifier', {
    'gamma': [8, 10, 12, 14],
    'learning_rate': [0.08, 0.1, 0.12],                
    'max_depth': [5, 10, 20, 30],
}))
# 1st search used 250k samples and undersampling (locking learning rate).
# Best parameters: gamma=8, learning_rate=0.08, max_depth=20 => score: 0.6114179052808012

### Build Optimal Model

In [ ]:
# Best params from grid search
xgb_pipeline = create_pipeline(
    XGBClassifier(
        gamma=8,
        learning_rate=0.08,
        max_depth=22,
    ), 
    sampling_strategy=SamplingStrategy.UNDERSAMPLING, 
    _y=y
)
# 'test_f1_macro': 0.6275257513386355, 'test_f1_micro': 0.6937688605366897

### Evaulate

In [ ]:
with timing():
    scores, averages = evaluate_model(X, y, xgb_pipeline)
    print(scores)
    print("Averages:", averages)

    confusionMatrix(X, y, xgb_pipeline, XGBOOST_VISUALIZATION_PATH)

### Training

In [ ]:
with timing():
    plot_learning_curve(X, y, xgb_pipeline, XGBOOST_VISUALIZATION_PATH)

### Save

In [ ]:
model_name = create_model_name()
pickle.dump(xgb_pipeline, open('%s/%s' % (XGBOOST_MODEL_PATH, model_name), 'wb'))